###konlpy, 라이브러리 설치
###데이터 불러오기

In [ ]:
# konlpy 설치
!pip install konlpy

     |████████████████████████████████| 19.4 MB 465 kB/s 
     |████████████████████████████████| 448 kB 48.6 MB/s 


In [ ]:
# 라이브러리

from konlpy.tag import Okt # komoran, hannanum, kkma, mecab
okt = Okt()
import os

import numpy as np
import pandas as pd

from datetime import datetime
import json
import re


from tqdm.notebook import tqdm

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
data = pd.read_excel('/content/크롤링.xlsx')
data

,위원회,사건번호,판정일,결과,내용
0,11,1999부해21,9199-05-31,NaN,NaN
1,10,2021부해337,2022-04-13,전부인정,"원심 의결서와 재심 의결서의 결정의 구체적 내용에 차이가 확인되는 점, 이 사건..."
2,12,2022부해26,2022-04-06,전부인정,가. 징계처분 1의 정당성 근로자가 2차례(2021. 5. 24. 및 2021....
3,1,2022부해272,2022-04-01,전부인정,가. 해고의 존재 여부 사용자가 해고의 의사표시를 하였고 근로자의 요청으로 근로계...
4,1,2022부해266,2022-03-31,각하,① 근로자는 사용자가 사용하는 상시근로자 수가 5인 이상이었다는 근거자료로 2021...
...,...,...,...,...,...
55705,2,2019부해522,--,각하,NaN
55706,0,2019부해1411,--,초심취소,NaN
55707,1,2019부해2972,--,기각,NaN
55708,8,2019부해623,--,기각,NaN


### 데이터 전처리


####제공되지 않은 내용 값(NaN) 날리기

In [ ]:
# 55,710개 -> 43,688개

new_data = data.dropna(inplace=False)
data['위원회'].count(), new_data['위원회'].count()

(55710, 43688)

####위원회 코드 0 제외 (중앙노동위원회 - 2심 담당)

In [ ]:
new_data = new_data[new_data['위원회']>0]
new_data

,위원회,사건번호,판정일,결과,내용
1,10,2021부해337,2022-04-13,전부인정,"원심 의결서와 재심 의결서의 결정의 구체적 내용에 차이가 확인되는 점, 이 사건..."
2,12,2022부해26,2022-04-06,전부인정,가. 징계처분 1의 정당성 근로자가 2차례(2021. 5. 24. 및 2021....
3,1,2022부해272,2022-04-01,전부인정,가. 해고의 존재 여부 사용자가 해고의 의사표시를 하였고 근로자의 요청으로 근로계...
4,1,2022부해266,2022-03-31,각하,① 근로자는 사용자가 사용하는 상시근로자 수가 5인 이상이었다는 근거자료로 2021...
5,11,2022부해18,2022-03-31,기각,"1. 상시근로자 수가 5명 이상인지 여부 ① 사업주가 서로 다른 개인인 점, ② ..."
...,...,...,...,...,...
53698,3,2014부노17,--,기각,가. 부당해고 구제실익이 있는지 여부 사용자가 근로자와의 촉탁근로계약을...
54445,2,2001부해192,--,각하,부 산 지 방 노 동 위 원 회 결 정 서2001부해19...
54447,9,2001부해53,--,기각,"강박된 상태에서 사직서를 제출하지 아니 하였고, 사직서가 제출되면 수리될 수도 있음..."
54482,2,2001부해120,--,전부인정,[피신청인 성명]전 정 도[피신청인 사업체명]성진지오텍(주)[신청요지]1. 본건 신...


####1차 전처리 (특수문자 등 날리기)

In [ ]:
new_data['내용'] = [re.sub('[^A-Za-z0-9가-힣]', ' ', s)for s in new_data['내용']]
new_data.reset_index(inplace=True)
new_data

,level_0,index,위원회,사건번호,판정일,결과,내용
0,0,1,10,2021부해337,2022-04-13,전부인정,원심 의결서와 재심 의결서의 결정의 구체적 내용에 차이가 확인되는 점 이 사건...
1,1,2,12,2022부해26,2022-04-06,전부인정,가 징계처분 1의 정당성 근로자가 2차례 2021 5 24 및 2021 ...
2,2,3,1,2022부해272,2022-04-01,전부인정,가 해고의 존재 여부 사용자가 해고의 의사표시를 하였고 근로자의 요청으로 근로계...
3,3,4,1,2022부해266,2022-03-31,각하,근로자는 사용자가 사용하는 상시근로자 수가 5인 이상이었다는 근거자료로 2021...
4,4,5,11,2022부해18,2022-03-31,기각,1 상시근로자 수가 5명 이상인지 여부 사업주가 서로 다른 개인인 점 ...
...,...,...,...,...,...,...,...
31288,31288,53698,3,2014부노17,--,기각,가 부당해고 구제실익이 있는지 여부 사용자가 근로자와의 촉탁근로계약을...
31289,31289,54445,2,2001부해192,--,각하,부 산 지 방 노 동 위 원 회 결 정 서2001부해19...
31290,31290,54447,9,2001부해53,--,기각,강박된 상태에서 사직서를 제출하지 아니 하였고 사직서가 제출되면 수리될 수도 있음...
31291,31291,54482,2,2001부해120,--,전부인정,피신청인 성명 전 정 도 피신청인 사업체명 성진지오텍 주 신청요지 1 본건 신...


####판결 라벨링

  (기각, 전부인정, 각하만)


In [ ]:
new_data.loc[new_data['결과'].str.contains('기각'), '라벨링'] = 0
new_data.loc[new_data['결과'].str.contains('전부인정'), '라벨링'] = 1
new_data.loc[new_data['결과'].str.contains('각하'), '라벨링'] = 2

new_data.dropna(inplace=True)

In [ ]:
# 라벨링 정수화
new_data['라벨링'] = new_data['라벨링'].astype(int)

In [ ]:
new_data.reset_index(drop=True,inplace=True)

In [ ]:
new_data

,level_0,index,위원회,사건번호,판정일,결과,내용,라벨링
0,0,1,10,2021부해337,2022-04-13,전부인정,원심 의결서와 재심 의결서의 결정의 구체적 내용에 차이가 확인되는 점 이 사건...,1
1,1,2,12,2022부해26,2022-04-06,전부인정,가 징계처분 1의 정당성 근로자가 2차례 2021 5 24 및 2021 ...,1
2,2,3,1,2022부해272,2022-04-01,전부인정,가 해고의 존재 여부 사용자가 해고의 의사표시를 하였고 근로자의 요청으로 근로계...,1
3,3,4,1,2022부해266,2022-03-31,각하,근로자는 사용자가 사용하는 상시근로자 수가 5인 이상이었다는 근거자료로 2021...,2
4,4,5,11,2022부해18,2022-03-31,기각,1 상시근로자 수가 5명 이상인지 여부 사업주가 서로 다른 개인인 점 ...,0
...,...,...,...,...,...,...,...,...
30145,31288,53698,3,2014부노17,--,기각,가 부당해고 구제실익이 있는지 여부 사용자가 근로자와의 촉탁근로계약을...,0
30146,31289,54445,2,2001부해192,--,각하,부 산 지 방 노 동 위 원 회 결 정 서2001부해19...,2
30147,31290,54447,9,2001부해53,--,기각,강박된 상태에서 사직서를 제출하지 아니 하였고 사직서가 제출되면 수리될 수도 있음...,0
30148,31291,54482,2,2001부해120,--,전부인정,피신청인 성명 전 정 도 피신청인 사업체명 성진지오텍 주 신청요지 1 본건 신...,1


####50글자 미만 내용 없애기(필요한 정보만 뽑아내기)

In [ ]:
fin_data = {
    '내용':[],
    '라벨':[]
}

In [ ]:
for i in tqdm(range(len(new_data))) :
  if len(list(new_data['내용'].values)[i]) >= 50 :
    fin_data['내용'].append(new_data['내용'][i])
    fin_data['라벨'].append(new_data['라벨링'][i])

fin_data = pd.DataFrame(fin_data)
fin_data

  0%|          | 0/30150 [00:00<?, ?it/s]

,내용,라벨
0,원심 의결서와 재심 의결서의 결정의 구체적 내용에 차이가 확인되는 점 이 사건...,1
1,가 징계처분 1의 정당성 근로자가 2차례 2021 5 24 및 2021 ...,1
2,가 해고의 존재 여부 사용자가 해고의 의사표시를 하였고 근로자의 요청으로 근로계...,1
3,근로자는 사용자가 사용하는 상시근로자 수가 5인 이상이었다는 근거자료로 2021...,2
4,1 상시근로자 수가 5명 이상인지 여부 사업주가 서로 다른 개인인 점 ...,0
...,...,...
29580,본 건 신청에 대하여 우리위원회는 당사자들의 주장 심문내용 관련자료 ...,0
29581,가 부당해고 구제실익이 있는지 여부 사용자가 근로자와의 촉탁근로계약을...,0
29582,부 산 지 방 노 동 위 원 회 결 정 서2001부해19...,2
29583,강박된 상태에서 사직서를 제출하지 아니 하였고 사직서가 제출되면 수리될 수도 있음...,0


####2차 전처리 (포스)
- 명사+동사만 추출

In [ ]:
def tokenizer(raw, pos=["Noun", "Verb" ] , stopword = []):
    return [
        word for word, tag in okt.pos(
            raw,
            norm=True,
            stem=True
        )
        if len(word) > 1 and tag in pos and word not in stopword
        ]

In [ ]:
okt.tagset

{'Adjective': '형용사',
 'Adverb': '부사',
 'Alpha': '알파벳',
 'Conjunction': '접속사',
 'Determiner': '관형사',
 'Eomi': '어미',
 'Exclamation': '감탄사',
 'Foreign': '외국어, 한자 및 기타기호',
 'Hashtag': '트위터 해쉬태그',
 'Josa': '조사',
 'KoreanParticle': '(ex: ㅋㅋ)',
 'Noun': '명사',
 'Number': '숫자',
 'PreEomi': '선어말어미',
 'Punctuation': '구두점',
 'ScreenName': '트위터 아이디',
 'Suffix': '접미사',
 'Unknown': '미등록어',
 'Verb': '동사'}

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer ,CountVectorizer

vectorize = TfidfVectorizer(
    tokenizer=tokenizer,
    min_df=2,
    sublinear_tf = True
)

In [ ]:
pos_data = {
    '내용':[],
     '라벨':[]
}
for i in tqdm(range(len(fin_data))):
  pos_data['내용'].append(tokenizer(fin_data['내용'][i]))
  pos_data['라벨'].append(fin_data['라벨'][i])


  0%|          | 0/29585 [00:00<?, ?it/s]

In [ ]:
df_pos_data=pd.DataFrame(pos_data)
df_pos_data

,내용,라벨
0,"[원심, 의결, 오다, 재심, 의결, 결정, 구체, 내용, 차이, 확인, 되다, 사...",1
1,"[가다, 징계처분, 정당성, 근로자, 차례, 교통사고, 내다, 징계, 사유, 해당,...",1
2,"[가다, 해고, 존재, 여부, 사용자, 해고, 의사표시, 하다, 근로자, 요청, 근...",1
3,"[근로자, 사용자, 사용, 하다, 근로자, 수가, 이상, 이다, 근거, 자료, 사용...",2
4,"[근로자, 수가, 이상, 여부, 사업, 주가, 서로, 다른, 개인, 업종, 다른, ...",0
...,...,...
29580,"[보다, 신청, 대하, 우리, 위원회, 당사자, 주장, 심문, 내용, 관련, 자료,...",0
29581,"[가다, 제실, 여부, 사용자, 근로자, 촉탁, 근로, 계약, 결정, 하다, 따르다...",0
29582,"[지다, 서다, 불다, 울산, 광역시, 울주군, 청량면, 남다, 신청, 울산, 광역...",2
29583,"[강박, 되다, 상태, 사직서, 제출, 하다, 하다, 사직서, 제출, 되다, 수리,...",0


In [ ]:
df_pos_data.to_excel('pos.xlsx', index=False) # 임시 저장

####3차 전처리 (각 행 [], 없애기 - 리스트 형태 해제)
  - 한 글자만 나오는 제외를 위함

In [ ]:
df_pos_data['내용'] = df_pos_data['내용'].apply(lambda x: ' '.join(x))
df_pos_data

,내용,라벨
0,원심 의결 오다 재심 의결 결정 구체 내용 차이 확인 되다 사건 사용자 대한 비위 ...,1
1,가다 징계처분 정당성 근로자 차례 교통사고 내다 징계 사유 해당 단체 협약 취업규칙...,1
2,가다 해고 존재 여부 사용자 해고 의사표시 하다 근로자 요청 근로 계약 종료 변경 ...,1
3,근로자 사용자 사용 하다 근로자 수가 이상 이다 근거 자료 사용자 계좌 이체 제출 ...,2
4,근로자 수가 이상 여부 사업 주가 서로 다른 개인 업종 다른 업무 근로 형태 다른 ...,0
...,...,...
29580,보다 신청 대하 우리 위원회 당사자 주장 심문 내용 관련 자료 판단 기초 하다 살피...,0
29581,가다 제실 여부 사용자 근로자 촉탁 근로 계약 결정 하다 따르다 복직 명령 촉탁 근...,0
29582,지다 서다 불다 울산 광역시 울주군 청량면 남다 신청 울산 광역시 중구 학성동 유진...,2
29583,강박 되다 상태 사직서 제출 하다 하다 사직서 제출 되다 수리 되다 수도 진술 사업...,0


####4차 전처리 (한 글지 날리기)

In [ ]:
# 한 글자만 남기는 함수 정의

def remove(x) :
    m = re.sub(r'(?:\b[0-9a-zA-Zㄱ-ㅎㅏ-ㅣ]\b|[?!\W]+)\s*', ' ', x).strip() 
    m = " ".join(m.split())
    return m

In [ ]:
f_data = {'내용' : [],
            '라벨' : []}

for i in tqdm(range(len(df_pos_data))) :
  f_data['내용'].append(remove(str(df_pos_data['내용'][i])))
  f_data['라벨'].append(df_pos_data['라벨'][i])

f_data = pd.DataFrame(f_data)
f_data

  0%|          | 0/29585 [00:00<?, ?it/s]

,내용,라벨
0,원심 의결 오다 재심 의결 결정 구체 내용 차이 확인 되다 사건 사용자 대한 비위 ...,1
1,가다 징계처분 정당성 근로자 차례 교통사고 내다 징계 사유 해당 단체 협약 취업규칙...,1
2,가다 해고 존재 여부 사용자 해고 의사표시 하다 근로자 요청 근로 계약 종료 변경 ...,1
3,근로자 사용자 사용 하다 근로자 수가 이상 이다 근거 자료 사용자 계좌 이체 제출 ...,2
4,근로자 수가 이상 여부 사업 주가 서로 다른 개인 업종 다른 업무 근로 형태 다른 ...,0
...,...,...
29580,보다 신청 대하 우리 위원회 당사자 주장 심문 내용 관련 자료 판단 기초 하다 살피...,0
29581,가다 제실 여부 사용자 근로자 촉탁 근로 계약 결정 하다 따르다 복직 명령 촉탁 근...,0
29582,지다 서다 불다 울산 광역시 울주군 청량면 남다 신청 울산 광역시 중구 학성동 유진...,2
29583,강박 되다 상태 사직서 제출 하다 하다 사직서 제출 되다 수리 되다 수도 진술 사업...,0


####stopword 계산을 위한 단어 빈도수 사전

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cvect = CountVectorizer() 
dtm = cvect.fit_transform(f_data['내용'])
dtm

<29585x17244 sparse matrix of type '<class 'numpy.int64'>'
	with 1708579 stored elements in Compressed Sparse Row format>

In [ ]:
feature_names = cvect.get_feature_names_out()
feature_names

array(['가가', '가감', '가게', ..., '힐튼', '힘내다', '힘쓰다'], dtype=object)

In [ ]:
vectorizer = CountVectorizer(analyzer = 'word', 
                             tokenizer = None, 
                             preprocessor = None, 
                             stop_words = None, 
                             min_df = 1,  
                             ngram_range=(1, 1), 
                             max_features = 300000
                            )
vectorizer

CountVectorizer(max_features=300000)

In [ ]:
feature_vector = vectorizer.fit_transform(f_data['내용'])
feature_vector.shape

(29585, 17244)

In [ ]:
vocab = vectorizer.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
dist = np.sum(feature_vector, axis=0)
    
df_freq = pd.DataFrame(dist, columns=vocab) 
df_freq_T = df_freq.T.sort_values(by=0, ascending=False)

In [ ]:
df_freq_T.reset_index(inplace=True)
df_freq_T.columns = ['단어','횟수']
df_freq_T

,단어,횟수
0,하다,248067
1,근로자,127856
2,사건,93910
3,사용자,72951
4,되다,66774
...,...,...
17239,목숨,1
17240,자괴,1
17241,자공시,1
17242,자공고,1


In [ ]:
# 엑셀로 저장

f_data.to_excel('f_data.xlsx', index=False)
df_freq_T.to_excel('df_freq_T.xlsx', index=False)

### stopword 돌리기

In [ ]:
data_bin = pd.read_excel('/content/df_freq_T.xlsx')
data_bin

,단어,횟수
0,하다,248067
1,근로자,127856
2,사건,93910
3,사용자,72951
4,되다,66774
...,...,...
17239,목숨,1
17240,자괴,1
17241,자공시,1
17242,자공고,1


In [ ]:
# 3번 이하 등장하는 단어들 삭제

data_count = data_bin.loc[data_bin['횟수']<=3]
data_count_bin = list(data_count['단어'].values)
data_count_bin

In [ ]:
# 전처리 데이터 입력
data = pd.read_excel('/content/f_data.xlsx')
data

,내용,라벨
0,원심 의결 오다 재심 의결 결정 구체 내용 차이 확인 되다 사건 사용자 대한 비위 ...,1
1,가다 징계처분 정당성 근로자 차례 교통사고 내다 징계 사유 해당 단체 협약 취업규칙...,1
2,가다 해고 존재 여부 사용자 해고 의사표시 하다 근로자 요청 근로 계약 종료 변경 ...,1
3,근로자 사용자 사용 하다 근로자 수가 이상 이다 근거 자료 사용자 계좌 이체 제출 ...,2
4,근로자 수가 이상 여부 사업 주가 서로 다른 개인 업종 다른 업무 근로 형태 다른 ...,0
...,...,...
29580,보다 신청 대하 우리 위원회 당사자 주장 심문 내용 관련 자료 판단 기초 하다 살피...,0
29581,가다 제실 여부 사용자 근로자 촉탁 근로 계약 결정 하다 따르다 복직 명령 촉탁 근...,0
29582,지다 서다 불다 울산 광역시 울주군 청량면 남다 신청 울산 광역시 중구 학성동 유진...,2
29583,강박 되다 상태 사직서 제출 하다 하다 사직서 제출 되다 수리 되다 수도 진술 사업...,0


In [ ]:
# 임의로 리스트화한 stopword에 3번 이하로 나온 단어들 추가

stopword = ['으로','에게','하고','여부','되어다','에서','대한','이다','대하','대해','라고','관련','에도','이라고','개월','부터','로서','에는','이고','또한','까지','이나','따라서','장하나','또는','로부터','거나','것이므','으로서','라는',2012,'자로','와의',2013,'통해',2011,'하나',2010,2009,'만으로','정이',2019,2018,'장이','이며','으로써',2008,'것임','하므다','가지','이라',2007,2021,2014,'으로부터','과의',2017,2015,2016,2020,'비다',00,'그러나','항제',24,'이로','각각','에서도','반면','에게는',000,'한편','역시','호에','에서의','까지의','000원','까지로','와는','먼저','해보다','라도','으로서의','일련','에게만','oo','어느','로기','로써','으로의','그리고','것일','부로','첫째','결코','2008년','2009년','그로','2010년','2007년','ooo','2005','20일','8월','10월','11월','이지만','만으로도','000만원','이라고는','0000','나름','00000','그냥','부터의','이라고도','이므로','경에','까지에','이냐','다소간','게다가','00조','이사나','사회자']
 
for i in data_count_bin:
  stopword.append(i)

In [ ]:
# 명사와 동사만 남기기

def tokenizer1(raw, pos=["Noun","Verb"] , stopword = stopword):
    return [
        word for word, tag in okt.pos(
            raw,
            norm=True,
            stem=True
        )
        if len(word) > 1 and tag in pos and word not in stopword
        ]


In [ ]:
N_V_data = {'내용':[],
            '라벨':[]}

for i in tqdm(range(len(data))) :
  N_V_data['내용'].append(tokenizer1(data['내용'][i]))
  N_V_data['라벨'].append(data['라벨'][i])

  0%|          | 0/29585 [00:00<?, ?it/s]

In [ ]:
# 명사 동사 데이터 저장

N_V_data = pd.DataFrame(N_V_data)
N_V_data['내용'] = N_V_data['내용'].apply(lambda x: ' '.join(x))
N_V_data.to_excel('Noun_Verb_data.xlsx', index=False)
N_V_data

,내용,라벨
0,원심 의결 오다 재심 의결 결정 구체 내용 차이 확인 되다 사건 사용자 비위 행위 ...,1
1,가다 징계처분 정당성 근로자 차례 교통사고 내다 징계 사유 해당 단체 협약 취업규칙...,1
2,가다 해고 존재 사용자 해고 의사표시 하다 근로자 요청 근로 계약 종료 변경 하다 ...,1
3,근로자 사용자 사용 하다 근로자 수가 이상 근거 자료 사용자 계좌 이체 제출 하다 ...,2
4,근로자 수가 이상 사업 주가 서로 다른 개인 업종 다른 업무 근로 형태 다른 사업 ...,0
...,...,...
29580,보다 신청 우리 위원회 당사자 주장 심문 내용 자료 판단 기초 하다 살피 보다 가다...,0
29581,가다 제실 사용자 근로자 촉탁 근로 계약 결정 하다 따르다 복직 명령 촉탁 근로 계...,0
29582,지다 서다 불다 울산 광역시 남다 신청 울산 광역시 중구 개발 대표이사 지다 대리인...,2
29583,강박 되다 상태 사직서 제출 하다 하다 사직서 제출 되다 수리 되다 수도 진술 사업...,0


In [ ]:
# 명사만 남기기
# 우리가 분석 타겟으로 정한 것

def tokenizer2(raw, pos=["Noun"] , stopword = stopword):
    return [
        word for word, tag in okt.pos(
            raw,
            norm=True,
            stem=True
        )
        if len(word) > 1 and tag in pos and word not in stopword
        ]


In [ ]:
V_data = {'내용' : [],
          '라벨' : []}

for i in tqdm(range(len(data))) :
  V_data['내용'].append(tokenizer2(data['내용'][i]))
  V_data['라벨'].append(data['라벨'][i])



  0%|          | 0/29585 [00:00<?, ?it/s]

In [ ]:
# 명사 데이터 저장

V_data = pd.DataFrame(V_data)
V_data['내용'] = V_data['내용'].apply(lambda x: ' '.join(x))
V_data.to_excel('Noun_data.xlsx',index=False)
V_data

,내용,라벨
0,원심 의결 재심 의결 결정 구체 내용 차이 확인 사건 사용자 비위 행위 인사 위원회...,1
1,징계처분 정당성 근로자 차례 교통사고 징계 사유 해당 단체 협약 취업규칙 징계 위원...,1
2,해고 존재 사용자 해고 의사표시 근로자 요청 근로 계약 종료 변경 근로자 사직서 작...,1
3,근로자 사용자 사용 근로자 수가 이상 근거 자료 사용자 계좌 이체 제출 해당 일정 ...,2
4,근로자 수가 이상 사업 주가 서로 다른 개인 업종 다른 업무 근로 형태 다른 사업 ...,0
...,...,...
29580,신청 우리 위원회 당사자 주장 심문 내용 자료 판단 기초 살피 신청 신청 계약해지 ...,0
29581,제실 사용자 근로자 촉탁 근로 계약 결정 복직 명령 촉탁 근로 계약 체결 근로자 체...,0
29582,지다 울산 광역시 신청 울산 광역시 중구 개발 대표이사 지다 대리인 노무사 공인 노...,2
29583,강박 상태 사직서 제출 사직서 제출 수리 수도 진술 사업 사직 종용 마음 사직서 제...,0
